<center> <h1>SEIR model</h1> </center>


SEIR model is one of the compartmental models which is used in mathematical modelling of infectious diseases. Here, the population is assigned to compartments with labels:
- S : Susceptible 
- E : Exposed 
- I : Infectious
- R : Recovered 

S is the fraction people who are able to contract the disease. E is the fraction of people who have been infected but are not yet infectious. I is the fraction of people who are capable of transmitting the disease. R is the fraction of people who have been recovered and have become immune. 

The number of people in S, E, I, R combine to form the population N. 
- $S + E + I + R = N$

Model:  
\begin{align*}
\mathrm{S} \overset{\beta }{\longrightarrow} \mathrm{E} \overset{\gamma}{\longrightarrow} \mathrm{I} \overset{\alpha}{\longrightarrow} \mathrm{R}  \\
\end{align*}

The infectious rate, $\mathrm{\beta}$, controls the rate of spread which represents the probability of transmitting disease between a susceptible and an exposed individual. The incubation rate,$\mathrm{\gamma}$, is the rate of latent individuals becoming infectious (average duration of incubation is 1/$\mathrm{\gamma}$). Recovery rate, $\mathrm{\alpha}$ is the rate at which people recover.

Ordinary Differential Equation (ODE):  
\begin{align*}
& \frac{\mathrm{d}S}{\mathrm{d}T}= -\beta S I  \\
& \frac{\mathrm{d}I}{\mathrm{d}T}= \beta S I - \gamma E  \\
& \frac{\mathrm{d}I}{\mathrm{d}T}= -\gamma E - \alpha I  \\
& \frac{\mathrm{d}R}{\mathrm{d}T}= \alpha I  \\
\end{align*}




**Background**: When you first get infected with a disease often you might not have symptoms and may not be spreading. This is the latency period. 

**Importing necessary libraries**:
1. Numpy: NumPy offers comprehensive mathematical functions, random number generators, linear algebra routines, Fourier transforms, and more. [Link](https://numpy.org/)
2. scipy: SciPy provides algorithms for optimization, integration, interpolation, eigenvalue problems, algebraic equations, differential equations, statistics and many more. [Link](https://scipy.org/)
3. scipy.integrate: The scipy.integrate sub-package provides several integration techniques including an ordinary differential equation integrator. [Link](https://docs.scipy.org/doc/scipy/tutorial/integrate.html)
4. matplotlib: Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python. [Link](https://matplotlib.org/)
5. scipy.optimize: SciPy optimize provides functions for minimizing (or maximizing) objective functions, possibly subject to constraints.[Link](https://docs.scipy.org/doc/scipy/reference/optimize.html)
6. csv: The csv module implements classes to read and write tabular data in CSV format. [Link](https://docs.python.org/3/library/csv.html)

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import csv
%matplotlib inline

Here N is the total population which is 1000. E0 which are the initially exposed which are 0. We have 10 infected people I0 and 990 susceptible people S0. 

We see that, S + E + I + R =  990 + 0 + 10 + 0 = 1000 = N, which is the total population. 

We have assumed beta which is the 

In [ ]:
N = 1000              # total population
E0 = 0                # Exposed
I0 = 10               # Infectious
R0 = 0                # Recovered
S0 = 990              # Susceptible

# S0 = N - I0 - E0 - R0
init_cond = S0, E0, I0, R0 # Initial conditions
Fitted_Parameters = 0.0001,0.2,0.03 # Fitted parameters (beta, gamma and alpha)
Initial_Guess = 0.0001,0.2,0.03    # Initial guess for our parameters

# Transition Rates
# beta = 0.5            # Disease transmission between Susceptible and exposed
# gamma = 0.3           # Disease transmission between exposed and infectious
# alpha = 0.3           # Recovery rate

# Calculating over the course of one year
daily = np.linspace(0, 365, 365)  # Returns evenly spaced numbers over an interval 0 to 365
tspan = daily

# Convering the ode model to a function
# The function SEIR_model takes in y, t, N, beta, gamma and alpha as it's inputs
    # y is the array of initial conditions - [S0, R0, I0, R0]
    # t is the array of time
    # N is the total population 
    # beta, gamma and alpha are the transition rates
    

# Initializing all of our parameters
Number_Parameters = len(Fitted_Parameters)   # Number of parameters 
ARE = np.zeros((6 , Number_Parameters), dtype=float) # Storage for Average Relative Estimation error
EstiParam = np.zeros((Number_Parameters), dtype = float) # Storage for estimated parameters
InfectData = np.zeros((len(tspan)), dtype = float)
ExitFlag = np.zeros((1), dtype = int) # Storage for exitflag
Fval = np.zeros((1), dtype = float) # Storage for optimization values 

# This function returns the system of ode's which we will solve using the odeint function
def SEIR_model(y, t, beta, gamma, alpha):
    S, E, I, R = y
    dSdt = -beta*S*I           #dS
    dEdt = beta*S*I - gamma*E  #dE
    dIdt = gamma*E - alpha*I          #dI
    dRdt = alpha*I                    #dR
    return dSdt, dEdt, dIdt, dRdt

y_est = odeint(SEIR_model, init_cond, daily, args = Fitted_Parameters)

In [ ]:
# The function err_in_dataSEIR takes in par which are the SEIR_model arguments and the optimized data
# The function returns the error 
def err_in_dataSEIR(par, odata):
    N = 1000              # total population
    E0 = 0                # Exposed
    I0 = 10               # Infectious
    R0 = 0                # Recovered
    S0 = N - I0 - E0 - R0 # Susceptible
    init_cond = [S0, E0, I0, R0] # Array of intial conditions (number of susceptible, exposed, infected and recovered people)
    
    EndofTime = 365       # Yearly    
    daily = np.linspace(1, EndofTime, EndofTime) #  Returns evenly spaced numbers over an interval 0 to 365
    tspan = daily 
    
    y = odeint(SEIR_model, init_cond, daily, args = (par[0],par[1],par[2]))
    # Here we are solving the system of equations that we modelled earlier
    # SEIR_model is the function that returns derivative values at requested y and t values as dydt
    # init_cond are the initial conditions of the differential states
    # t is the time points at which the solutions should be reported
    # arguments are the input values that our model needs (extra arguments to pass to the function)
    
    Infected = y[:,2] # third column of the solution corresponding to infected
    weight = 1/np.square(Infected)
    error_in_data = np.sum(weight*(Infected - odata)**2) # finding the error
    
    return error_in_data
    
    

In [ ]:
column_number = 0                                      # Column number for the first case is 0 since we we want the data in the first column 
arr = []                                               # Initializing an empty array so we can append more data into it 
with open('Infected data_1.csv', newline='') as f:     # Opening our csv file
    reader = csv.reader(f)                             # Reading the csv, csv.reader automatically reads the csv row wise
    for row in reader:                                 # Since we need only the first element of each row we append the required
        arr.append(float(row[column_number]))          # csv.reader reads in the data in the data type string - so we convert it to float 
        
arr = np.array(arr)                                    # Converting the array into numpy array, as numpy arrays are faster
Prevalencedata = arr

optimizer0 = minimize(err_in_dataSEIR, Initial_Guess, args=Prevalencedata , method='Nelder-Mead') # Calling the optimization function
EstiParam = optimizer0.x                    # Estimated Parameters 
print(EstiParam.T)
exitflag = optimizer0.success               # Exit flag (either 1 or 0)
fval = optimizer0.fun                       # Value of the function
message = optimizer0.message                # Reason attached to the exit flag
if exitflag==0:
    print(message)
    
# Computing the ARE score
ARE_Value = np.array(np.zeros((Number_Parameters), dtype = float)) 


for i in range(Number_Parameters):
    ARE_Value[i] = (100) * np.sum(abs(Fitted_Parameters[i] - EstiParam[i]))/ abs(Fitted_Parameters[i])

[ 2.19798637e-04  3.25773200e-01 -3.51964862e-02]


## Plot the noisy data and the optimised model below!